In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17080, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17080 (delta 20), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17080/17080), 15.69 MiB | 25.42 MiB/s, done.
Resolving deltas: 100% (11725/11725), done.


In [2]:
%cd yolov5
!pip install -r requirements.txt

/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.3/904.3 kB 17.7 MB/s eta 0:00:00a 0:00:01


In [3]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  

/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-12-27 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 152MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
!pip install effdet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 4.6 MB/s eta 0:00:00


In [5]:
import random
from pycocotools.coco import COCO
from torchvision.transforms import Compose, Normalize
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class CocoDataset(Dataset):
    def __init__(self, root, annotation_file, transforms=None, image_size=(512, 512), subset_size=None):
        self.root = root
        self.coco = COCO(annotation_file)
        self.ids = list(self.coco.imgs.keys())

        if subset_size:
            self.ids = random.sample(self.ids, subset_size)

        self.transforms = transforms
        self.image_size = image_size

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        image_id = self.ids[idx]
        annotation_ids = self.coco.getAnnIds(imgIds=image_id)
        annotations = self.coco.loadAnns(annotation_ids)

        valid_annotations = [
            ann for ann in annotations if ann["bbox"] and ann["bbox"][2] > 0 and ann["bbox"][3] > 0
        ]

        if len(valid_annotations) == 0: 
            return None

        image_info = self.coco.imgs[image_id]
        image_path = os.path.join(self.root, image_info['file_name'])
        img = Image.open(image_path).convert("RGB")

        img = img.resize(self.image_size, Image.Resampling.LANCZOS)

        boxes = []
        labels = []
        for ann in valid_annotations:
            x, y, width, height = ann['bbox']
            x_scale = self.image_size[1] / image_info['width']
            y_scale = self.image_size[0] / image_info['height']
            boxes.append([
                x * x_scale, y * y_scale,
                (x + width) * x_scale, (y + height) * y_scale
            ])
            labels.append(ann['category_id'])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        targets = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([image_id])
        }

        if self.transforms:
            img = self.transforms(img)

        return img, targets

def collate_fn(batch):
    batch = [sample for sample in batch if sample is not None]  
    return tuple(zip(*batch))

def get_coco_data_loader(root, annotation_file, batch_size=4, model_type="fasterrcnn", image_size=(512, 512), subset_size=None):
    transforms = Compose([
        TF.to_tensor,
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    dataset = CocoDataset(root=root, annotation_file=annotation_file, transforms=transforms, image_size=image_size, subset_size=subset_size)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from tqdm import tqdm
from effdet import create_model as EfficientDet
import torchvision.transforms.functional as TF
import os
import matplotlib.pyplot as plt
import random
import warnings
import torch.nn.functional as F

os.environ['PYTHONWARNINGS'] = 'ignore'
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="torch.cuda.amp")
warnings.filterwarnings("ignore", category=FutureWarning, module=".*ultralytics.*")


class UniversalDomainGeneralizationModule(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.feature_size = None  
        self.domain_discriminator = None  

    def forward(self, x, domain_label=None, alpha=1.0):
        if isinstance(x, list):
            x = torch.stack(x, dim=0)

        features = self.backbone(x)
        if isinstance(features, (tuple, list)):
            combined_features = torch.cat([feat.flatten(1) for feat in features], dim=1)
        elif isinstance(features, dict):
            combined_features = torch.cat([feat.flatten(1) for feat in features.values()], dim=1)
        else:
            combined_features = features.flatten(1)

        if self.feature_size is None:
            self.feature_size = combined_features.size(1)
            self.domain_discriminator = nn.Sequential(
                nn.Linear(self.feature_size, 128),
                nn.ReLU(),
                nn.Linear(128, 1),
                nn.Sigmoid()
            )
            self.domain_discriminator.to(combined_features.device)

        if domain_label is not None:
            reversed_features = GradientReversalLayer.apply(combined_features, alpha)
            domain_preds = self.domain_discriminator(reversed_features)
            return combined_features, domain_preds
        return combined_features, None


class GradientReversalLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.alpha * grad_output, None

from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn

class FasterRCNNWithUDGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.faster_rcnn = fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)  
        self.udgm = UniversalDomainGeneralizationModule(self.faster_rcnn.backbone)

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            images = torch.stack(images, dim=0)

        features, domain_preds = self.udgm(images, domain_label, alpha)
        if targets is not None:
            outputs = self.faster_rcnn(images, targets)
            return outputs, domain_preds
        outputs = self.faster_rcnn(images)
        return outputs, domain_preds

class YOLOv5WithUDGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        self.udgm = UniversalDomainGeneralizationModule(self.yolov5.model.model)

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            images = torch.stack(images, dim=0)  

        features, domain_preds = self.udgm(images, domain_label, alpha)
        predictions = self.yolov5(images)

        return {'predictions': predictions}, domain_preds


class EfficientDetWithUDGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.effdet = EfficientDet('tf_efficientdet_d0', pretrained=True, num_classes=80)
        self.udgm = UniversalDomainGeneralizationModule(self.effdet.backbone)

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            images = torch.stack(images, dim=0)

        features, domain_preds = self.udgm(images, domain_label, alpha)
        class_outputs, box_outputs = self.effdet(images)

        if targets is not None:
            classification_loss, regression_loss = compute_efficientdet_loss(
                (class_outputs, box_outputs), targets, num_classes=80
            )
            total_loss = classification_loss + regression_loss
            return {
                'classification_loss': classification_loss,
                'regression_loss': regression_loss,
                'total_loss': total_loss,
            }, domain_preds

        return (class_outputs, box_outputs), domain_preds

class UnifiedModelWithUDGM(nn.Module):
    def __init__(self, model_type):
        super().__init__()
        self.model_type = model_type.lower()

        if self.model_type == "fasterrcnn":
            self.model = FasterRCNNWithUDGM()
        elif self.model_type == "yolov5":
            self.model = YOLOv5WithUDGM()
        elif self.model_type == "efficientdet":
            self.model = EfficientDetWithUDGM()
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            images = [img if isinstance(img, torch.Tensor) else TF.to_tensor(img) for img in images]

        if self.model_type in ["fasterrcnn", "efficientdet"]:
            return self.model(images, targets, domain_label, alpha)
        elif self.model_type == "yolov5":
            predictions, domain_preds = self.model(images, targets, domain_label, alpha)
            outputs = {"predictions": predictions}  
            return outputs, domain_preds
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

def train(model, train_loader, optimizer, device, criterion=None):
    model.train()
    optimizer.zero_grad()

    for images, targets in tqdm(train_loader, desc="Training Loop (1 Epoch)"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        torch.cuda.empty_cache()

        domain_label = torch.ones(len(images), device=device)

        outputs, domain_preds = model(images, targets, domain_label=domain_label)

        if model.model_type == "fasterrcnn":
            detection_loss = sum(loss for loss in outputs.values()) if isinstance(outputs, dict) else outputs.loss
        elif model.model_type == "efficientdet":
            detection_loss = outputs['total_loss']
        elif model.model_type == "yolov5":
            predictions = outputs['predictions']  
            detection_loss = predictions.get('loss', torch.tensor(0.0, device=device))

        domain_loss = criterion(domain_preds, domain_label.view(-1, 1)) if criterion and domain_preds is not None else 0

        total_loss = detection_loss + domain_loss
        total_loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        torch.cuda.empty_cache()

def compute_efficientdet_loss(predictions, targets, num_classes=80):
    cls_loss = 0.0
    box_loss = 0.0

    pred_classes, pred_boxes = predictions

    for pred_cls, pred_box, target in zip(pred_classes, pred_boxes, targets):
        gt_boxes = target['boxes']
        gt_labels = target['labels']

        matched_cls_targets, matched_box_targets, positive_indices = match_anchors(
            pred_box.view(-1, 4), gt_boxes, gt_labels, num_classes
        )

        matched_cls_targets = matched_cls_targets.view_as(pred_cls)
        pred_cls = pred_cls.view(-1, num_classes)
        matched_cls_targets = matched_cls_targets.view(-1, num_classes)

        cls_loss += focal_loss(pred_cls, matched_cls_targets)

        if positive_indices.sum() > 0:
            pred_boxes_pos = pred_box.view(-1, 4)[positive_indices]
            matched_boxes_pos = matched_box_targets[positive_indices]
            box_loss += nn.SmoothL1Loss()(pred_boxes_pos, matched_boxes_pos)

    total_loss = (cls_loss + box_loss) / len(targets)
    return cls_loss, box_loss

def match_anchors(pred_boxes, gt_boxes, gt_labels, num_classes):
    iou_matrix = compute_iou(pred_boxes, gt_boxes)
    max_iou, matched_indices = iou_matrix.max(dim=1)

    positive_indices = max_iou > 0.5
    negative_indices = max_iou < 0.4

    matched_cls_targets = torch.zeros((pred_boxes.size(0), num_classes), device=pred_boxes.device)

    if positive_indices.any():
        matched_gt_labels = gt_labels[matched_indices[positive_indices]].long()
        matched_cls_targets[positive_indices, matched_gt_labels] = 1.0

    matched_box_targets = torch.zeros_like(pred_boxes)
    if positive_indices.any():
        matched_box_targets[positive_indices] = gt_boxes[matched_indices[positive_indices]]

    return matched_cls_targets, matched_box_targets, positive_indices

def decode_boxes(pred_box_encodings, anchors):
    if pred_box_encodings.size(0) != anchors.size(0):
        raise ValueError(
            f"Mismatch in sizes: pred_box_encodings ({pred_box_encodings.size(0)}) "
            f"and anchors ({anchors.size(0)})"
        )

    wa = anchors[:, 2] - anchors[:, 0]
    ha = anchors[:, 3] - anchors[:, 1]
    xa = anchors[:, 0] + 0.5 * wa
    ya = anchors[:, 1] + 0.5 * ha

    tx, ty, tw, th = pred_box_encodings[:, 0], pred_box_encodings[:, 1], pred_box_encodings[:, 2], pred_box_encodings[:, 3]
    wg = torch.exp(tw) * wa
    hg = torch.exp(th) * ha
    xg = tx * wa + xa
    yg = ty * ha + ya

    x_min = xg - 0.5 * wg
    y_min = yg - 0.5 * hg
    x_max = xg + 0.5 * wg
    y_max = yg + 0.5 * hg

    decoded = torch.stack([x_min, y_min, x_max, y_max], dim=1)
    return decoded

def generate_anchors(num_anchors_per_cell, grid_size=(32, 32), batch_size=1):
    grid_height, grid_width = grid_size
    cell_size = 1.0 / grid_height

    anchors = []
    for y in range(grid_height):
        for x in range(grid_width):
            center_x = (x + 0.5) * cell_size
            center_y = (y + 0.5) * cell_size
            for scale in [0.1, 0.2, 0.3]:  
                for aspect_ratio in [0.5, 1.0, 2.0]:
                    w = scale * aspect_ratio
                    h = scale / aspect_ratio
                    x_min = center_x - w / 2
                    y_min = center_y - h / 2
                    x_max = center_x + w / 2
                    y_max = center_y + h / 2
                    anchors.append([x_min, y_min, x_max, y_max])

    anchors = torch.tensor(anchors, device="cuda").view(-1, 4)
    anchors = anchors.unsqueeze(0).repeat(batch_size, 1, 1) 
    return anchors

def flatten_box_outputs(box_outputs):
    batch_size, num_anchors_per_cell, height, width = box_outputs.shape
    flattened = box_outputs.permute(0, 2, 3, 1).reshape(batch_size, -1, 4)
    return flattened

def focal_loss(inputs, targets, alpha=0.25, gamma=2.0):
    probs = torch.sigmoid(inputs)
    ce_loss = nn.BCEWithLogitsLoss(reduction="none")(inputs, targets)
    pt = torch.where(targets == 1, probs, 1 - probs)
    loss = alpha * (1 - pt) ** gamma * ce_loss
    return loss.sum()

def evaluate(model, val_loader, device, visualize_samples=5):
    model.eval()
    total_iou = 0
    total_predictions = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    iou_threshold = 0.5

    visualized = 0

    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(tqdm(val_loader, desc="Evaluation Loop")):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs, _ = model(images)

            for i, target in enumerate(targets):
                if model.model_type == "efficientdet":
                    if isinstance(outputs, tuple) and len(outputs) == 2:
                        class_outputs, box_outputs = outputs
                        
                    flattened_box_outputs = torch.cat(
                        [b.permute(0, 2, 3, 1).reshape(b.size(0), -1, 4) for b in box_outputs], dim=1
                    )

                    anchors = torch.cat(
                        [
                            generate_anchors(
                                num_anchors_per_cell=3,
                                grid_size=(b.size(2), b.size(3)),
                                batch_size=b.size(0),
                            )
                            for b in box_outputs
                        ],
                        dim=1,
                    )
                    if flattened_box_outputs.size(1) == anchors.size(1):  
                        pred_boxes = decode_boxes(flattened_box_outputs.view(-1, 4), anchors.view(-1, 4)).detach()
                    else:
                        raise ValueError(
                            f"Mismatch in sizes: box_outputs ({flattened_box_outputs.size(1)}) and anchors ({anchors.size(1)})"
                        )

                    pred_scores = torch.ones(pred_boxes.size(0), device=device)
                    pred_labels = torch.zeros(pred_boxes.size(0), dtype=torch.int64, device=device)
                        
                elif model.model_type == "fasterrcnn":
                    pred_boxes = outputs[i]['boxes'].detach()
                    pred_scores = outputs[i]['scores'].detach()
                    pred_labels = outputs[i]['labels'].detach()

                elif model.model_type == "yolov5":
                    predictions = outputs['predictions']['predictions']
                    pred_boxes = predictions[0][:, :4].detach() 
                    pred_scores = predictions[0][:, 4].detach()
                    pred_labels = predictions[0][:, 5:].argmax(dim=1).detach()

                true_boxes = target['boxes']
                true_labels = target['labels']

                iou_matrix = compute_iou(pred_boxes, true_boxes)

                if iou_matrix.numel() > 0:
                    max_iou_per_pred, matched_gt_idx = iou_matrix.max(dim=1)
                    matched_gt_idx[max_iou_per_pred < iou_threshold] = -1
                    matched_gt_mask = matched_gt_idx >= 0

                    true_positives += matched_gt_mask.sum().item()
                    false_positives += (~matched_gt_mask).sum().item()
                    false_negatives += (true_labels.size(0) - matched_gt_mask.sum().item())

                    total_iou += max_iou_per_pred[matched_gt_mask].sum().item()
                    total_predictions += matched_gt_mask.sum().item()

    mean_iou = total_iou / total_predictions if total_predictions > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    mAP = (precision + recall) / 2 if (precision + recall) > 0 else 0
    wada = (mean_iou + mAP) / 2

    return mAP, wada, mean_iou

def compute_iou(pred_boxes, true_boxes):
    if pred_boxes.size(0) == 0 or true_boxes.size(0) == 0:
        return torch.zeros((pred_boxes.size(0), true_boxes.size(0)), device=pred_boxes.device)

    true_boxes = true_boxes.to(pred_boxes.device)

    inter_xmin = torch.max(pred_boxes[:, None, 0], true_boxes[:, 0])
    inter_ymin = torch.max(pred_boxes[:, None, 1], true_boxes[:, 1])
    inter_xmax = torch.min(pred_boxes[:, None, 2], true_boxes[:, 2])
    inter_ymax = torch.min(pred_boxes[:, None, 3], true_boxes[:, 3])

    inter_area = (inter_xmax - inter_xmin).clamp(0) * (inter_ymax - inter_ymin).clamp(0)

    pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
    true_area = (true_boxes[:, 2] - true_boxes[:, 0]) * (true_boxes[:, 3] - true_boxes[:, 1])

    union_area = pred_area[:, None] + true_area - inter_area

    iou = inter_area / union_area.clamp(min=1e-6)
    return iou

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "fasterrcnn"  

model = UnifiedModelWithUDGM(model_type=model_type).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

train_dataset_path = "/kaggle/input/coco-2017-dataset/coco2017/train2017"  
val_dataset_path = "/kaggle/input/coco-2017-dataset/coco2017/val2017"      

train_annotations = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json" 
val_annotations = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json"

train_loader = get_coco_data_loader(
    root=train_dataset_path,
    annotation_file=train_annotations,
    batch_size=4,
    model_type=model_type,
    image_size=(512, 512),
    subset_size=500
)

val_loader = get_coco_data_loader(
    root=val_dataset_path,
    annotation_file=val_annotations,
    batch_size=4,
    model_type=model_type,
    image_size=(512, 512),
    subset_size=100
)

print(f"-----{model_type.upper()}-----")

print("Starting Training")
train(model, train_loader, optimizer, device, criterion)

if model_type == "efficientdet":
    pass
else:
    print("Starting Evaluation")
    mAP, wada, mean_iou = evaluate(model, val_loader, device, visualize_samples=5)
    print(f"Evaluation Results: mAP={mAP:.4f}, WADA={wada:.4f}, Mean IoU={mean_iou:.4f}")
    
torch.save(model.state_dict(), f"{model_type}_udgm_model.pth")
print(f"Model saved as {model_type}_udgm_model.pth")

Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100%|██████████| 74.2M/74.2M [00:05<00:00, 13.6MB/s]


loading annotations into memory...
Done (t=18.59s)
creating index...
index created!
loading annotations into memory...
Done (t=0.83s)
creating index...
index created!
-----FASTERRCNN-----
Starting Training


Training Loop (1 Epoch): 100%|██████████| 125/125 [00:40<00:00,  3.07it/s]


Starting Evaluation


Evaluation Loop: 100%|██████████| 25/25 [00:05<00:00,  4.75it/s]


Evaluation Results: mAP=0.8191, WADA=0.7541, Mean IoU=0.6891
Model saved as fasterrcnn_udgm_model.pth


In [7]:
model_type = "yolov5"  

model = UnifiedModelWithUDGM(model_type=model_type).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

train_dataset_path = "/kaggle/input/coco-2017-dataset/coco2017/train2017"  
val_dataset_path = "/kaggle/input/coco-2017-dataset/coco2017/val2017"      

train_annotations = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json" 
val_annotations = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json"

train_loader = get_coco_data_loader(
    root=train_dataset_path,
    annotation_file=train_annotations,
    batch_size=4,
    model_type=model_type,
    image_size=(512, 512),
    subset_size=500
)

val_loader = get_coco_data_loader(
    root=val_dataset_path,
    annotation_file=val_annotations,
    batch_size=4,
    model_type=model_type,
    image_size=(512, 512),
    subset_size=100
)

print(f"-----{model_type.upper()}-----")

if model_type == "efficientdet":
    pass
else:
    print("Starting Evaluation")
    mAP, wada, mean_iou = evaluate(model, val_loader, device, visualize_samples=5)
    print(f"Evaluation Results: mAP={mAP:.4f}, WADA={wada:.4f}, Mean IoU={mean_iou:.4f}")
    
torch.save(model.state_dict(), f"{model_type}_udgm_model.pth")
print(f"Model saved as {model_type}_udgm_model.pth")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-27 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


loading annotations into memory...
Done (t=17.78s)
creating index...
index created!
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
-----YOLOV5-----
Starting Evaluation


Evaluation Loop: 100%|██████████| 25/25 [00:05<00:00,  4.61it/s]


Evaluation Results: mAP=0.1539, WADA=0.3683, Mean IoU=0.5827
Model saved as yolov5_udgm_model.pth


In [8]:
model_type = "efficientdet"  

model = UnifiedModelWithUDGM(model_type=model_type).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

train_dataset_path = "/kaggle/input/coco-2017-dataset/coco2017/train2017"  
val_dataset_path = "/kaggle/input/coco-2017-dataset/coco2017/val2017"      

train_annotations = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json" 
val_annotations = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json"

train_loader = get_coco_data_loader(
    root=train_dataset_path,
    annotation_file=train_annotations,
    batch_size=4,
    model_type=model_type,
    image_size=(512, 512),
    subset_size=500
)

val_loader = get_coco_data_loader(
    root=val_dataset_path,
    annotation_file=val_annotations,
    batch_size=4,
    model_type=model_type,
    image_size=(512, 512),
    subset_size=100
)

print(f"-----{model_type.upper()}-----")

print("Starting Training")
train(model, train_loader, optimizer, device, criterion)

if model_type == "efficientdet":
    pass
else:
    print("Starting Evaluation")
    mAP, wada, mean_iou = evaluate(model, val_loader, device, visualize_samples=5)
    print(f"Evaluation Results: mAP={mAP:.4f}, WADA={wada:.4f}, Mean IoU={mean_iou:.4f}")

torch.save(model.state_dict(), f"{model_type}_udgm_model.pth")
print(f"Model saved as {model_type}_udgm_model.pth")

Downloading: "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_d0_34-f153e0cf.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientdet_d0_34-f153e0cf.pth


loading annotations into memory...
Done (t=15.72s)
creating index...
index created!
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!
-----EFFICIENTDET-----
Starting Training


Training Loop (1 Epoch): 100%|██████████| 125/125 [01:01<00:00,  2.03it/s]


Model saved as efficientdet_udgm_model.pth
